In [1]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
import numpy as np
from pprint import pprint
from tqdm.auto import tqdm
from tqdm import tqdm_notebook
from scipy.sparse import hstack
from collections import defaultdict
tqdm.pandas()

In [2]:
df_description = pd.read_csv('./data/data_definition.txt', sep='\t')

df_train_genba = pd.read_csv('./data/train_genba.tsv', sep='\t')
df_train_goto = pd.read_csv('./data/train_goto.tsv', sep='\t')

df_train = df_train_goto.merge(df_train_genba, on='pj_no', how='left')
df_train.drop('id', axis=1, inplace=True)

df_test_genba = pd.read_csv('./data/test_genba.tsv', sep='\t')
df_test_goto = pd.read_csv('./data/test_goto.tsv', sep='\t')

df_test = df_test_goto.merge(df_test_genba, on='pj_no', how='left')
test_surface = df_test['tc_mseki']
df_test.drop('id', axis=1, inplace=True)

## Preprocessing

In [3]:
def fill_city_name(name):
    if '市' not in name and '郡' not in name:
        name = '市' + name
    return name

def split_address(df):
    df['jukyo'] = df['jukyo'].str.slice(start=3).str.replace(r'[ヶｹ]', 'ケ')
    df['jukyo'] = df['jukyo'].apply(fill_city_name)
    city_split = df['jukyo'].str.split(r'[市郡]', n=1, expand=True)
    df['city'] = city_split[0]
    street_split = city_split[1].str.split(r'[町区]', n=1, expand=True)
    df['street'] = street_split[0]
    df['address_detail'] = street_split[1].str.strip().replace('', None)
    return df

df_train = split_address(df_train)
df_test = split_address(df_test)

df_train.drop(['kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)
df_test.drop(['kaoku_um', 'toshikuiki2', 'shu_sogi'], axis=1, inplace=True)

In [4]:
def combine(row, combine_list, tup):
    l = set()
    for col in tup:
        if pd.notnull(row[col]):
            l.add(row[col])
    combine_list.append(','.join(l))


combine_cols = [('yoto', 100), ('road_hk', 100)]
for i, tup in enumerate([['yoto1', 'yoto2'], ['road1_hk', 'road2_hk', 'road3_hk', 'road4_hk']]):
    combine_train = []
    combine_test = []
    
    combine_col_name = combine_cols[i][0]
    _ = df_train.apply(lambda row: combine(row, combine_train, tup), axis=1)
    _ = df_test.apply(lambda row: combine(row, combine_test, tup), axis=1)

    count_vectorizer = CountVectorizer(min_df=combine_cols[i][1])
    combine_train_matrix = count_vectorizer.fit_transform(combine_train).todense()
    combine_test_matrix = count_vectorizer.transform(combine_test).todense()
    for i in range(combine_train_matrix.shape[1]):
        df_train['%s_%d' % (combine_col_name, i)] = combine_train_matrix[:, i]
        df_test['%s_%d' % (combine_col_name, i)] = combine_test_matrix[:, i]
    for col in tup:
        df_train.drop(col, axis=1, inplace=True)
        df_test.drop(col, axis=1, inplace=True)

In [5]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
price_stats = []
for train_idx, valid_idx in splitter.split(df_train):
    price_stats_by_city = defaultdict(dict)
    for city, group in df_train.iloc[train_idx].groupby('city'):
        price_list = group['keiyaku_pr']/group['tc_mseki']
        price_stats_by_city[city]['price_by_city_mean'] = price_list.mean()
        price_stats_by_city[city]['price_by_city_median'] = price_list.median()
        price_stats_by_city[city]['price_by_city_min'] = price_list.min()
        price_stats_by_city[city]['price_by_city_max'] = price_list.max()
        price_stats_by_city[city]['price_by_city_std'] = price_list.std()
        price_stats_by_city[city]['price_by_city_count'] = len(price_list)
    for i, city in enumerate(df_train.iloc[valid_idx]['city']):
        price_stats.append((valid_idx[i], price_stats_by_city[city]))

price_stats_test = [] 
price_stats_by_city = defaultdict(dict)
for city, group in df_train.groupby('city'):
    price_list = group['keiyaku_pr']/group['tc_mseki']
    price_stats_by_city[city]['price_by_city_mean'] = price_list.mean()
    price_stats_by_city[city]['price_by_city_median'] = price_list.median()
    price_stats_by_city[city]['price_by_city_min'] = price_list.min()
    price_stats_by_city[city]['price_by_city_max'] = price_list.max()
    price_stats_by_city[city]['price_by_city_std'] = price_list.std()
    price_stats_by_city[city]['price_by_city_count'] = len(price_list)
for city in df_test['city']:
    price_stats_test.append(price_stats_by_city[city])
    
df_price_stats = pd.DataFrame([x[1] for x in sorted(price_stats, key=lambda x: x[0])])
df_price_stats_test = pd.DataFrame(price_stats_test)

df_train = pd.concat([df_train, df_price_stats], axis=1)
df_test = pd.concat([df_test, df_price_stats_test], axis=1)

In [6]:
for col in ['mseki_rd_hb', 'road3_fi', 'rosenka_hb']:
    df_train[col].replace(0.0, np.nan, inplace=True)
    df_test[col].replace(0.0, np.nan, inplace=True)

## Train

In [7]:
continue_features = list(df_description[(df_description['データ型'] == '数値') & (df_description['項目名'] != 'pj_no')]['項目名'])
continue_features += ['price_by_city_mean', 'price_by_city_median', 'price_by_city_min', 'price_by_city_max', 
                      'price_by_city_std', 'price_by_city_count']
objective = 'keiyaku_pr'
categorical_features = list(df_train)

for f in continue_features+[objective]:
    if f in categorical_features:
        categorical_features.remove(f)

for col in categorical_features:
    dt = df_train[col].dtype
    if dt == int or dt == float:
        df_train[col].fillna(-1, inplace=True)
        df_test[col].fillna(-1, inplace=True)
    else:
        df_train[col].fillna('', inplace=True)
        df_test[col].fillna('', inplace=True)
        
for col in categorical_features:
    if col not in ['pj_no']:
        vc = dict(df_train[col].value_counts())
        for k, v in vc.items():
            if v <= 3:
                df_train[col].replace(k, 'Low_freq', inplace=True)
                df_test[col].replace(k, 'Low_freq', inplace=True)
        le = LabelEncoder()
        df_train[col] = le.fit_transform(df_train[col])
        mapping = {v: i for i, v in enumerate(le.classes_)}
        df_test[col] = df_test[col].apply(lambda x: mapping[x] if x in mapping else -1)
        
for col in continue_features:
    if col != 'keiyaku_pr':
        scaler = MinMaxScaler()
        df_train[col] = scaler.fit_transform(df_train[col].values.reshape(-1, 1))
        df_test[col] = scaler.transform(df_test[col].values.reshape(-1, 1))
    
df_test['keiyaku_pr'] = 0

## LGB

In [8]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    X_train, y_train = train.drop('keiyaku_pr', axis=1), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = valid.drop('keiyaku_pr', axis=1), np.log(valid['keiyaku_pr']+1)
    regressor = lgb.LGBMRegressor(n_estimators=20000, learning_rate=0.01, silent=False, random_state=28,
                                 bagging_fraction=0.9, bagging_freq=1, feature_fraction=0.8, lambda_l1=0.01)
    regressor.fit(X_train, y_train, eval_set=(X_valid, y_valid), early_stopping_rounds=500,
                  categorical_feature=categorical_features)
    prediction_list.append(regressor.predict(df_test[df_train.drop(objective, axis=1).columns]))
    best_scores.append(regressor.best_score_['valid_0']['l2'])

print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission.tsv', sep='\t', header=None, index=False)

# 0.01209830

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['address_detail', 'bas_toho1', 'bas_toho2', 'bastei_nm1', 'bastei_nm2', 'bokachiiki', 'bus_yohi', 'chikukeikaku', 'city', 'eki_nm1', 'eki_nm2', 'fi3m_yohi', 'fi4m_yohi', 'gas', 'gesui', 'hiatari', 'hokakisei1', 'hokakisei2', 'hokakisei3', 'hokakisei4', 'hokakyoka', 'hw_status', 'jigata', 'josui', 'jukyo', 'kaihatsukyoka', 'kborjs', 'keikakuroad', 'kinshijiko', 'kobetsu1', 'kobetsu2', 'kobetsu3', 'kobetsu4', 'kodochiku', 'levelplan', 'pj_no', 'road1_sb', 'road2_sb', 'road3_sb', 'road4_sb', 'road_hk_0', 'road_hk_1', 'road_hk_2', 'road_hk_3', 'road_st', 'rosen_nm1', 'rosen_nm2', 'rs_e_kdate2', 'rs_e_kdate3', 'rs_e_m_ari', 'rs_e_m_nashi', 'rs_e_parking', 'rs_e_tahata', 'rs_e_zoki', 'rs_n_kdate2', 'rs_n_kdate3', 'rs_n_m_ari', 'rs_n_m_nashi', 'rs_n_parking', 'rs_n_tahata', 'rs_n_zoki', 'rs_s_kdate2', 'rs_s_kdate3', 'rs_s_m_ari', 'rs_s_m_nashi',

[1]	valid_0's l2: 0.0676468
Training until validation scores don't improve for 500 rounds.
[2]	valid_0's l2: 0.0669477
[3]	valid_0's l2: 0.0661976
[4]	valid_0's l2: 0.0654715
[5]	valid_0's l2: 0.0647454
[6]	valid_0's l2: 0.0640629
[7]	valid_0's l2: 0.0634036
[8]	valid_0's l2: 0.0627194
[9]	valid_0's l2: 0.062045
[10]	valid_0's l2: 0.0614324
[11]	valid_0's l2: 0.0607899
[12]	valid_0's l2: 0.0601706
[13]	valid_0's l2: 0.0595516
[14]	valid_0's l2: 0.0589893
[15]	valid_0's l2: 0.0584024
[16]	valid_0's l2: 0.0578338
[17]	valid_0's l2: 0.0572569
[18]	valid_0's l2: 0.0567087
[19]	valid_0's l2: 0.0561493
[20]	valid_0's l2: 0.0556355
[21]	valid_0's l2: 0.0550979
[22]	valid_0's l2: 0.0545749
[23]	valid_0's l2: 0.0540725
[24]	valid_0's l2: 0.0535759
[25]	valid_0's l2: 0.0530661
[26]	valid_0's l2: 0.0525757
[27]	valid_0's l2: 0.052082
[28]	valid_0's l2: 0.0515915
[29]	valid_0's l2: 0.0511212
[30]	valid_0's l2: 0.0506526
[31]	valid_0's l2: 0.0502192
[32]	valid_0's l2: 0.0497829
[33]	valid_0's l2: 0

[327]	valid_0's l2: 0.0201362
[328]	valid_0's l2: 0.0201167
[329]	valid_0's l2: 0.0201016
[330]	valid_0's l2: 0.0200859
[331]	valid_0's l2: 0.0200712
[332]	valid_0's l2: 0.020057
[333]	valid_0's l2: 0.0200406
[334]	valid_0's l2: 0.0200238
[335]	valid_0's l2: 0.02001
[336]	valid_0's l2: 0.0199921
[337]	valid_0's l2: 0.0199764
[338]	valid_0's l2: 0.0199618
[339]	valid_0's l2: 0.0199447
[340]	valid_0's l2: 0.0199355
[341]	valid_0's l2: 0.0199238
[342]	valid_0's l2: 0.0199078
[343]	valid_0's l2: 0.0198905
[344]	valid_0's l2: 0.0198738
[345]	valid_0's l2: 0.0198618
[346]	valid_0's l2: 0.0198489
[347]	valid_0's l2: 0.0198343
[348]	valid_0's l2: 0.0198214
[349]	valid_0's l2: 0.0198119
[350]	valid_0's l2: 0.0197955
[351]	valid_0's l2: 0.0197817
[352]	valid_0's l2: 0.019772
[353]	valid_0's l2: 0.0197588
[354]	valid_0's l2: 0.0197467
[355]	valid_0's l2: 0.0197339
[356]	valid_0's l2: 0.0197192
[357]	valid_0's l2: 0.0197072
[358]	valid_0's l2: 0.0196978
[359]	valid_0's l2: 0.0196907
[360]	valid_0'

[600]	valid_0's l2: 0.0179831
[601]	valid_0's l2: 0.0179803
[602]	valid_0's l2: 0.0179782
[603]	valid_0's l2: 0.017977
[604]	valid_0's l2: 0.0179751
[605]	valid_0's l2: 0.0179772
[606]	valid_0's l2: 0.0179761
[607]	valid_0's l2: 0.0179748
[608]	valid_0's l2: 0.0179694
[609]	valid_0's l2: 0.0179627
[610]	valid_0's l2: 0.0179601
[611]	valid_0's l2: 0.0179567
[612]	valid_0's l2: 0.0179509
[613]	valid_0's l2: 0.0179457
[614]	valid_0's l2: 0.017944
[615]	valid_0's l2: 0.0179416
[616]	valid_0's l2: 0.0179372
[617]	valid_0's l2: 0.0179332
[618]	valid_0's l2: 0.0179269
[619]	valid_0's l2: 0.0179205
[620]	valid_0's l2: 0.0179179
[621]	valid_0's l2: 0.0179147
[622]	valid_0's l2: 0.0179135
[623]	valid_0's l2: 0.0179109
[624]	valid_0's l2: 0.0179035
[625]	valid_0's l2: 0.0179008
[626]	valid_0's l2: 0.0178946
[627]	valid_0's l2: 0.0178906
[628]	valid_0's l2: 0.0178866
[629]	valid_0's l2: 0.017881
[630]	valid_0's l2: 0.0178791
[631]	valid_0's l2: 0.0178761
[632]	valid_0's l2: 0.0178727
[633]	valid_0

[882]	valid_0's l2: 0.0173554
[883]	valid_0's l2: 0.017353
[884]	valid_0's l2: 0.0173515
[885]	valid_0's l2: 0.0173473
[886]	valid_0's l2: 0.0173449
[887]	valid_0's l2: 0.017344
[888]	valid_0's l2: 0.0173414
[889]	valid_0's l2: 0.0173408
[890]	valid_0's l2: 0.0173383
[891]	valid_0's l2: 0.0173384
[892]	valid_0's l2: 0.0173397
[893]	valid_0's l2: 0.0173384
[894]	valid_0's l2: 0.017336
[895]	valid_0's l2: 0.0173342
[896]	valid_0's l2: 0.0173318
[897]	valid_0's l2: 0.017332
[898]	valid_0's l2: 0.0173315
[899]	valid_0's l2: 0.0173301
[900]	valid_0's l2: 0.0173292
[901]	valid_0's l2: 0.0173295
[902]	valid_0's l2: 0.017329
[903]	valid_0's l2: 0.0173276
[904]	valid_0's l2: 0.017326
[905]	valid_0's l2: 0.0173277
[906]	valid_0's l2: 0.0173273
[907]	valid_0's l2: 0.0173286
[908]	valid_0's l2: 0.0173286
[909]	valid_0's l2: 0.017326
[910]	valid_0's l2: 0.0173243
[911]	valid_0's l2: 0.0173235
[912]	valid_0's l2: 0.0173239
[913]	valid_0's l2: 0.0173234
[914]	valid_0's l2: 0.0173211
[915]	valid_0's l

...

[2055]	valid_0's l2: 0.0114769
[2056]	valid_0's l2: 0.0114752
[2057]	valid_0's l2: 0.0114745
[2058]	valid_0's l2: 0.0114749
[2059]	valid_0's l2: 0.0114752
[2060]	valid_0's l2: 0.0114746
[2061]	valid_0's l2: 0.0114751
[2062]	valid_0's l2: 0.0114749
[2063]	valid_0's l2: 0.0114743
[2064]	valid_0's l2: 0.0114748
[2065]	valid_0's l2: 0.0114747
[2066]	valid_0's l2: 0.0114753
[2067]	valid_0's l2: 0.0114757
[2068]	valid_0's l2: 0.0114764
[2069]	valid_0's l2: 0.0114765
[2070]	valid_0's l2: 0.0114762
[2071]	valid_0's l2: 0.0114766
[2072]	valid_0's l2: 0.0114752
[2073]	valid_0's l2: 0.011475
[2074]	valid_0's l2: 0.011475
[2075]	valid_0's l2: 0.0114748
[2076]	valid_0's l2: 0.0114743
[2077]	valid_0's l2: 0.0114738
[2078]	valid_0's l2: 0.0114736
[2079]	valid_0's l2: 0.011474
[2080]	valid_0's l2: 0.0114739
[2081]	valid_0's l2: 0.011474
[2082]	valid_0's l2: 0.0114738
[2083]	valid_0's l2: 0.0114744
[2084]	valid_0's l2: 0.011474
[2085]	valid_0's l2: 0.0114742
[2086]	valid_0's l2: 0.0114741
[2087]	valid_

[2348]	valid_0's l2: 0.0114318
[2349]	valid_0's l2: 0.0114336
[2350]	valid_0's l2: 0.0114334
[2351]	valid_0's l2: 0.0114334
[2352]	valid_0's l2: 0.0114334
[2353]	valid_0's l2: 0.0114337
[2354]	valid_0's l2: 0.0114345
[2355]	valid_0's l2: 0.0114342
[2356]	valid_0's l2: 0.0114342
[2357]	valid_0's l2: 0.0114333
[2358]	valid_0's l2: 0.0114341
[2359]	valid_0's l2: 0.0114331
[2360]	valid_0's l2: 0.0114335
[2361]	valid_0's l2: 0.0114333
[2362]	valid_0's l2: 0.0114326
[2363]	valid_0's l2: 0.0114322
[2364]	valid_0's l2: 0.0114312
[2365]	valid_0's l2: 0.0114304
[2366]	valid_0's l2: 0.0114316
[2367]	valid_0's l2: 0.0114306
[2368]	valid_0's l2: 0.0114306
[2369]	valid_0's l2: 0.011431
[2370]	valid_0's l2: 0.0114321
[2371]	valid_0's l2: 0.0114322
[2372]	valid_0's l2: 0.0114334
[2373]	valid_0's l2: 0.0114331
[2374]	valid_0's l2: 0.0114334
[2375]	valid_0's l2: 0.0114332
[2376]	valid_0's l2: 0.0114329
[2377]	valid_0's l2: 0.0114326
[2378]	valid_0's l2: 0.0114327
[2379]	valid_0's l2: 0.0114331
[2380]	va

[2639]	valid_0's l2: 0.0114202
[2640]	valid_0's l2: 0.01142
[2641]	valid_0's l2: 0.0114193
[2642]	valid_0's l2: 0.0114194
[2643]	valid_0's l2: 0.0114203
[2644]	valid_0's l2: 0.0114205
[2645]	valid_0's l2: 0.0114212
[2646]	valid_0's l2: 0.0114211
[2647]	valid_0's l2: 0.0114214
[2648]	valid_0's l2: 0.0114218
[2649]	valid_0's l2: 0.011422
[2650]	valid_0's l2: 0.0114221
[2651]	valid_0's l2: 0.0114223
[2652]	valid_0's l2: 0.0114211
[2653]	valid_0's l2: 0.0114189
[2654]	valid_0's l2: 0.011418
[2655]	valid_0's l2: 0.0114169
[2656]	valid_0's l2: 0.0114175
[2657]	valid_0's l2: 0.0114178
[2658]	valid_0's l2: 0.0114172
[2659]	valid_0's l2: 0.0114174
[2660]	valid_0's l2: 0.0114168
[2661]	valid_0's l2: 0.0114164
[2662]	valid_0's l2: 0.0114161
[2663]	valid_0's l2: 0.0114167
[2664]	valid_0's l2: 0.011417
[2665]	valid_0's l2: 0.0114173
[2666]	valid_0's l2: 0.0114171
[2667]	valid_0's l2: 0.0114182
[2668]	valid_0's l2: 0.0114179
[2669]	valid_0's l2: 0.0114188
[2670]	valid_0's l2: 0.0114187
[2671]	valid_

[2940]	valid_0's l2: 0.0114009
[2941]	valid_0's l2: 0.0114016
[2942]	valid_0's l2: 0.011402
[2943]	valid_0's l2: 0.0114019
[2944]	valid_0's l2: 0.0114017
[2945]	valid_0's l2: 0.0114011
[2946]	valid_0's l2: 0.0114013
[2947]	valid_0's l2: 0.0114015
[2948]	valid_0's l2: 0.0114019
[2949]	valid_0's l2: 0.0114015
[2950]	valid_0's l2: 0.0113999
[2951]	valid_0's l2: 0.0114001
[2952]	valid_0's l2: 0.0114001
[2953]	valid_0's l2: 0.0114004
[2954]	valid_0's l2: 0.0114002
[2955]	valid_0's l2: 0.0114002
[2956]	valid_0's l2: 0.0114006
[2957]	valid_0's l2: 0.011401
[2958]	valid_0's l2: 0.0114011
[2959]	valid_0's l2: 0.0114016
[2960]	valid_0's l2: 0.0114019
[2961]	valid_0's l2: 0.0114011
[2962]	valid_0's l2: 0.0114005
[2963]	valid_0's l2: 0.011401
[2964]	valid_0's l2: 0.0114019
[2965]	valid_0's l2: 0.0114014
[2966]	valid_0's l2: 0.0114013
[2967]	valid_0's l2: 0.011401
[2968]	valid_0's l2: 0.0114012
[2969]	valid_0's l2: 0.0114008
[2970]	valid_0's l2: 0.0114013
[2971]	valid_0's l2: 0.011401
[2972]	valid_

[3234]	valid_0's l2: 0.0114031
[3235]	valid_0's l2: 0.0114033
[3236]	valid_0's l2: 0.0114033
[3237]	valid_0's l2: 0.011403
[3238]	valid_0's l2: 0.011403
[3239]	valid_0's l2: 0.0114036
[3240]	valid_0's l2: 0.0114033
[3241]	valid_0's l2: 0.011404
[3242]	valid_0's l2: 0.0114037
[3243]	valid_0's l2: 0.0114038
[3244]	valid_0's l2: 0.0114037
[3245]	valid_0's l2: 0.0114039
[3246]	valid_0's l2: 0.0114042
[3247]	valid_0's l2: 0.0114047
[3248]	valid_0's l2: 0.0114047
[3249]	valid_0's l2: 0.0114047
[3250]	valid_0's l2: 0.0114049
[3251]	valid_0's l2: 0.0114049
[3252]	valid_0's l2: 0.011404
[3253]	valid_0's l2: 0.0114043
[3254]	valid_0's l2: 0.011404
[3255]	valid_0's l2: 0.0114049
[3256]	valid_0's l2: 0.0114056
[3257]	valid_0's l2: 0.0114057
[3258]	valid_0's l2: 0.0114052
[3259]	valid_0's l2: 0.0114048
[3260]	valid_0's l2: 0.0114048
[3261]	valid_0's l2: 0.0114047
[3262]	valid_0's l2: 0.0114051
[3263]	valid_0's l2: 0.0114048
[3264]	valid_0's l2: 0.0114048
[3265]	valid_0's l2: 0.0114043
[3266]	valid_

## Ridge

In [9]:
from sklearn import linear_model

continue_features.remove('keiyaku_pr')
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
df_train.fillna(0, inplace=True)
df_test.fillna(0, inplace=True)
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    test = df_test[df_train.drop(objective, axis=1).columns]
    encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
    train_cat_one_hot = encoder.fit_transform(train[categorical_features])
    valid_cat_one_hot = encoder.transform(valid[categorical_features])
    test_cat_one_hot = encoder.transform(test[categorical_features])
    X_train, y_train = hstack([train[continue_features].values, train_cat_one_hot]), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = hstack([valid[continue_features].values, valid_cat_one_hot]), np.log(valid['keiyaku_pr']+1)
    regressor = linear_model.Ridge(alpha=2, random_state=28)
    regressor.fit(X_train, y_train)
    pred_val = regressor.predict(X_valid)
    prediction_list.append(regressor.predict(hstack([test[continue_features].values, test_cat_one_hot])))
    best_scores.append(mean_squared_error(y_valid, pred_val))
    
print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission_ridge.tsv', sep='\t', header=None, index=False)

# 0.01404797

5-fold cv mean l2 0.01404797


## HuberRegressor

In [10]:
splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    test = df_test[df_train.drop(objective, axis=1).columns]
    encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
    train_cat_one_hot = encoder.fit_transform(train[categorical_features])
    valid_cat_one_hot = encoder.transform(valid[categorical_features])
    test_cat_one_hot = encoder.transform(test[categorical_features])
    X_train, y_train = hstack([train[continue_features].values, train_cat_one_hot]), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = hstack([valid[continue_features].values, valid_cat_one_hot]), np.log(valid['keiyaku_pr']+1)
    regressor = linear_model.HuberRegressor(max_iter=500, epsilon=1.2, alpha=0.00001)
    regressor.fit(X_train, y_train)
    pred_val = regressor.predict(X_valid)
    prediction_list.append(regressor.predict(hstack([test[continue_features].values, test_cat_one_hot])))
    best_scores.append(mean_squared_error(y_valid, pred_val))
    
print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission_huber.tsv', sep='\t', header=None, index=False)

# 0.01360072

5-fold cv mean l2 0.01360072


## Linear svr

In [11]:
from sklearn import svm

splitter = KFold(n_splits=5, shuffle=True, random_state=28)
prediction_list = []
best_scores = []
for train_idx, valid_idx in splitter.split(df_train):
    train, valid = df_train.iloc[train_idx], df_train.iloc[valid_idx]
    test = df_test[df_train.drop(objective, axis=1).columns]
    encoder = OneHotEncoder(categories='auto', handle_unknown='ignore')
    train_cat_one_hot = encoder.fit_transform(train[categorical_features])
    valid_cat_one_hot = encoder.transform(valid[categorical_features])
    test_cat_one_hot = encoder.transform(test[categorical_features])
    X_train, y_train = hstack([train[continue_features].values, train_cat_one_hot]), np.log(train['keiyaku_pr']+1)
    X_valid, y_valid = hstack([valid[continue_features].values, valid_cat_one_hot]), np.log(valid['keiyaku_pr']+1)
    regressor = svm.LinearSVR(C=0.1, epsilon=0.01, random_state=28)
    regressor.fit(X_train, y_train)
    pred_val = regressor.predict(X_valid)
    prediction_list.append(regressor.predict(hstack([test[continue_features].values, test_cat_one_hot])))
    best_scores.append(mean_squared_error(y_valid, pred_val))
    
print("5-fold cv mean l2 %.8f" % np.mean(best_scores))

df_submission = pd.read_csv('./data/sample_submit.tsv', sep='\t', names=['id', 'pred'])

df_submission['pred'] = np.exp(np.mean(prediction_list, axis=0))-1
df_submission.to_csv('submission_linear_svr.tsv', sep='\t', header=None, index=False)

# 0.01325935

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


5-fold cv mean l2 0.01325935


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


## Merge

In [12]:
df_1 = pd.read_csv('./submission.tsv', sep='\t', names=['id', 'pred'])     #0.01209830
df_2 = pd.read_csv('./submission_ridge.tsv', sep='\t', names=['id', 'pred'])   #0.01404797
df_3 = pd.read_csv('./submission_huber.tsv', sep='\t', names=['id', 'pred'])  #0.01360072
df_4 = pd.read_csv('./submission_linear_svr.tsv', sep='\t', names=['id', 'pred'])  #0.01325935

df_1['pred'] = df_1['pred']*0.75 + df_2['pred']*0.025 + df_3['pred']*0.075 + df_4['pred']*0.15
df_1.to_csv('submission_merge.tsv', sep='\t', index=False, header=None) 